In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import data_process as dp
import validate as v

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

%matplotlib inline

In [2]:
df = pd.read_csv('FinalDataLabeled.csv',  index_col=0)
df

,Date,Price,RSI,MovingAverage10,EMA10,MovingAverage21,EMA21,MovingAverage50,EMA50,LowerBB,...,RoC12,Momentum14,RoC14,Momentum21,RoC21,AvgSentiment,TweetFreq,signals_7d,signals_14d,signals_30d
167,2011-01-01,0.300000,75.369476,0.275862,0.280994,0.258329,0.265562,0.253609,0.243577,0.213209,...,0.123596,1.244813,0.244813,1.315789,0.315789,1.0,0.000014,1.0,1.0,1.0
168,2011-01-02,0.299970,75.937629,0.280862,0.284444,0.262145,0.268690,0.254089,0.245788,0.216271,...,0.242441,1.249354,0.249354,1.364408,0.364408,0.5,0.000000,1.0,1.0,1.0
169,2011-01-03,0.295000,64.976230,0.285562,0.286363,0.265354,0.271082,0.254408,0.247718,0.217471,...,0.180000,1.104869,0.104869,1.296104,0.296104,0.5,0.000000,1.0,1.0,1.0
170,2011-01-04,0.298950,90.013927,0.290467,0.288652,0.267842,0.273616,0.255018,0.249727,0.219984,...,0.195943,1.238216,0.238216,1.211845,0.211845,0.5,0.000000,1.0,1.0,1.0
171,2011-01-05,0.298916,88.615791,0.293862,0.290518,0.270648,0.275916,0.256496,0.251656,0.221606,...,0.205308,1.195665,0.195665,1.245485,0.245485,0.5,0.000000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4180,2021-12-27,50796.377674,69.602868,49219.216563,49804.785557,48857.489121,50127.943013,54580.594563,52486.513401,45566.701358,...,0.040257,1.085733,0.085733,1.005164,0.005164,0.5,0.000000,NaN,NaN,NaN
4181,2021-12-28,47673.044045,47.384776,49296.033600,49417.196192,48719.325846,49904.770380,54183.220334,52297.749897,45497.817908,...,0.000201,0.987169,-0.012831,0.942631,-0.057369,0.5,0.000000,NaN,NaN,NaN
4182,2021-12-29,46470.321575,40.567501,49255.423065,48881.400807,48527.405237,49592.547761,53770.715052,52069.223296,45290.272181,...,0.002932,0.951664,-0.048336,0.920192,-0.079808,1.0,0.052500,NaN,NaN,NaN
4183,2021-12-30,47102.463004,47.657892,49270.193018,48557.957570,48487.626197,49366.176420,53417.642753,51874.448382,45246.062154,...,0.004213,0.988230,-0.011770,0.982574,-0.017426,1.0,0.349371,NaN,NaN,NaN


In [3]:
df_fil = df
df_fil['Date'] = pd.to_datetime(df_fil['Date'],format='%Y-%m-%d')
df_fil = df_fil[df_fil['Date'] >= pd.Timestamp(2015, 1, 1)]
df_fil = df_fil[df_fil['Date'] <= pd.Timestamp(2021, 12, 31)]
df_fil

,Date,Price,RSI,MovingAverage10,EMA10,MovingAverage21,EMA21,MovingAverage50,EMA50,LowerBB,...,RoC12,Momentum14,RoC14,Momentum21,RoC21,AvgSentiment,TweetFreq,signals_7d,signals_14d,signals_30d
1628,2015-01-01,314.776472,50.602414,319.924813,319.468805,327.067766,328.172198,353.887226,345.515290,300.070243,...,-0.046322,1.004034,0.004034,0.902991,-0.097009,1.0,0.008380,0.0,0.0,0.0
1629,2015-01-02,315.942732,48.865961,317.910673,318.827701,325.294057,327.060428,351.767791,344.355582,300.589318,...,-0.015732,0.992772,-0.007228,0.894539,-0.105461,1.0,0.007996,0.0,0.0,0.0
1630,2015-01-03,285.647310,31.474856,314.172175,312.794903,322.265693,323.295599,349.512959,342.053296,296.156665,...,-0.139208,0.865425,-0.134575,0.817904,-0.182096,1.0,0.007337,0.0,0.0,0.0
1631,2015-01-04,263.334575,28.344632,308.606018,303.802116,317.849258,317.844597,347.219774,338.966288,283.766133,...,-0.216462,0.820376,-0.179624,0.739538,-0.260462,1.0,0.009463,1.0,0.0,0.0
1632,2015-01-05,275.003852,28.782956,303.185369,298.566068,314.416337,313.949984,344.915735,336.457957,276.788537,...,-0.148680,0.828718,-0.171282,0.792301,-0.207699,1.0,0.007543,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4180,2021-12-27,50796.377674,69.602868,49219.216563,49804.785557,48857.489121,50127.943013,54580.594563,52486.513401,45566.701358,...,0.040257,1.085733,0.085733,1.005164,0.005164,0.5,0.000000,NaN,NaN,NaN
4181,2021-12-28,47673.044045,47.384776,49296.033600,49417.196192,48719.325846,49904.770380,54183.220334,52297.749897,45497.817908,...,0.000201,0.987169,-0.012831,0.942631,-0.057369,0.5,0.000000,NaN,NaN,NaN
4182,2021-12-29,46470.321575,40.567501,49255.423065,48881.400807,48527.405237,49592.547761,53770.715052,52069.223296,45290.272181,...,0.002932,0.951664,-0.048336,0.920192,-0.079808,1.0,0.052500,NaN,NaN,NaN
4183,2021-12-30,47102.463004,47.657892,49270.193018,48557.957570,48487.626197,49366.176420,53417.642753,51874.448382,45246.062154,...,0.004213,0.988230,-0.011770,0.982574,-0.017426,1.0,0.349371,NaN,NaN,NaN


In [4]:
x_train, x_test, y_train, y_test = dp.scale_split(df_fil, 30, 1)

In [5]:
from tpot import TPOTClassifier
from sklearn.neural_network import MLPClassifier

parameters = {'hidden_layer_sizes' : [(10,10),(20,20),(30,30),(40,40),(50,50),(80,80),(100,100)],
              'activation' : ['identity', 'logistic', 'tanh', 'relu'],
              'solver': ['lbfgs', 'sgd', 'adam'],
              'learning_rate': ['constant','invscaling','adaptive'],
              'alpha': [0.1,0.01,0.001,0.0001,0.00001],
              'max_iter': [500]}
               
tpot_classifier = TPOTClassifier(verbosity= 2, early_stop=5, n_jobs = -1,
                                 config_dict=
                                 {'sklearn.neural_network.MLPClassifier': parameters}, 
                                 cv = 4, scoring = 'accuracy')

tpot_classifier.fit(x_train,y_train) 

C:\Python38\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                                  
Generation 1 - Current best internal CV score: 0.8040572535514421
                                                                                  
Generation 2 - Current best internal CV score: 0.8040572535514421
                                                                                   
Generation 3 - Current best internal CV score: 0.8040572535514421
                                                                                   
Generation 4 - Current best internal CV score: 0.8040572535514421
                                                                                   
Generation 5 - Current best internal CV score: 0.8040572535514421
                                                                                   
Generation 6 - Current best internal CV score: 0.8040572535514421
                                                                                  
The optimized pipel

TPOTClassifier(config_dict={'sklearn.neural_network.MLPClassifier': {'activation': ['identity',
                                                                                    'logistic',
                                                                                    'tanh',
                                                                                    'relu'],
                                                                     'alpha': [0.1,
                                                                               0.01,
                                                                               0.001,
                                                                               0.0001,
                                                                               1e-05],
                                                                     'hidden_layer_sizes': [(10,
                                                                                             10),